In [1]:
from torch.utils.data import DataLoader
from resnet import ResNet
from sklearn.model_selection import train_test_split
import numpy as np
from custom_dataset import FinalDataset
import torch
from torch import nn

In [2]:
file_path= "D:\\thesis_data\\filteredData.hdf5"

In [3]:
dataset = FinalDataset(file_path)

In [4]:
total_size = len(dataset)
indices = np.arange(total_size)
np.random.shuffle(indices)
train_indices, test_indices = train_test_split(indices, test_size=0.2)

train_dataset = FinalDataset(file_path, indices=train_indices)
test_dataset = FinalDataset(file_path, indices=test_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle =True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [5]:
model = ResNet(12,24)

In [6]:
model.train()

ResNet(
  (Conv1): Conv1d(12, 24, kernel_size=(3,), stride=(1,), padding=(1,))
  (BatchNorm1): BatchNorm1d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ReLU1): ReLU()
  (ResBlocks): Sequential(
    (0): ResBlock(
      (MaxPool1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (Conv1_A): Conv1d(24, 48, kernel_size=(1,), stride=(1,))
      (Conv1_B): Conv1d(24, 48, kernel_size=(3,), stride=(2,))
      (BatchNorm1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ReLU1): ReLU()
      (Dropout1): Dropout(p=0.5, inplace=False)
      (Conv2_B): Conv1d(48, 48, kernel_size=(3,), stride=(1,), padding=(1,))
      (BatchNorm2): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ReLU2): ReLU()
      (Dropout2): Dropout(p=0.5, inplace=False)
    )
    (1): ResBlock(
      (MaxPool1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (

In [7]:
for id, (signal, label) in enumerate(train_loader):
    output = model(signal)
    print(output)
    break


torch.Size([32, 4000, 12])
torch.Size([32, 12, 4000])
torch.Size([32, 95616])
tensor([[0.6087],
        [0.5174],
        [0.4733],
        [0.4741],
        [0.5934],
        [0.3882],
        [0.9047],
        [0.6364],
        [0.4763],
        [0.5705],
        [0.4037],
        [0.5458],
        [0.5808],
        [0.3848],
        [0.5228],
        [0.7869],
        [0.5427],
        [0.6348],
        [0.4867],
        [0.3911],
        [0.8466],
        [0.4466],
        [0.4581],
        [0.4979],
        [0.4943],
        [0.6146],
        [0.5293],
        [0.5112],
        [0.4458],
        [0.4302],
        [0.4783],
        [0.5587]], grad_fn=<SigmoidBackward0>)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

for epoch in range(10):
    for id, (signal, label) in enumerate(train_loader):
        output = model(signal)
        output = output.squeeze()
        loss = loss_fn(output, label.float())
        loss.backward()
        optimizer.step()
        print(loss)
        break


torch.Size([32, 4000, 12])
torch.Size([32, 12, 4000])
torch.Size([32, 95616])


TypeError: 'torch.dtype' object is not callable